In [14]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
import os
import shutil
from math import ceil
from PIL import Image
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import CosineAnnealingLR

In [15]:
# Base EfficientNet architecture details
base_model = [
    [1, 16, 1, 1, 3],
    [6, 24, 2, 2, 3],
    [6, 40, 2, 2, 5],
    [6, 80, 3, 2, 3],
    [6, 112, 3, 1, 5],
    [6, 192, 4, 2, 5],
    [6, 320, 1, 1, 3],
]

phi_values = {
    "b0": (0, 224, 0.2),
    "b1": (0.5, 240, 0.2),
    "b2": (1, 260, 0.3),
    "b3": (2, 300, 0.3),
    "b4": (3, 380, 0.4),
    "b5": (4, 456, 0.4),
    "b6": (5, 528, 0.5),
    "b7": (6, 600, 0.5),
}

In [16]:
# EfficientNet-related building blocks
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, groups=1):
        super(CNNBlock, self).__init__()
        self.cnn = nn.Conv2d(
            in_channels, out_channels, kernel_size, stride, padding, groups=groups, bias=False
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.silu = nn.SiLU()

    def forward(self, x):
        return self.silu(self.bn(self.cnn(x)))

In [17]:
class SqueezeExcitation(nn.Module):
    def __init__(self, in_channels, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, reduced_dim, 1),
            nn.SiLU(),
            nn.Conv2d(reduced_dim, in_channels, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)


In [18]:
class CBAM(nn.Module):
    def __init__(self, channels, reduction=16):
        super(CBAM, self).__init__()
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, kernel_size=1, bias=False),
            nn.ReLU(),
            nn.Conv2d(channels // reduction, channels, kernel_size=1, bias=False),
            nn.Sigmoid(),
        )
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        ca = self.channel_attention(x)
        x = x * ca

        max_pool = torch.max(x, dim=1, keepdim=True)[0]
        avg_pool = torch.mean(x, dim=1, keepdim=True)
        sa = torch.cat([max_pool, avg_pool], dim=1)
        sa = self.spatial_attention(sa)
        x = x * sa

        return x

In [19]:
class InvertedResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, expand_ratio, reduction=4, survival_prob=0.8):
        super(InvertedResidualBlock, self).__init__()
        self.survival_prob = survival_prob
        self.use_residual = in_channels == out_channels and stride == 1
        self.hidden_dim = in_channels * expand_ratio
        self.expand = expand_ratio != 1

        if self.expand:
            self.expand_conv = CNNBlock(in_channels, self.hidden_dim, kernel_size=3, stride=1, padding=1)

        self.conv = nn.Sequential(
            CNNBlock(self.hidden_dim, self.hidden_dim, kernel_size, stride, padding, groups=self.hidden_dim),
            SqueezeExcitation(self.hidden_dim, reduced_dim=int(self.hidden_dim / reduction)),
            nn.Conv2d(self.hidden_dim, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
        )

        self.cbam = CBAM(out_channels)

    def stochastic_depth(self, x):
        if not self.training:
            return x
        binary_tensor = (
            torch.rand(x.shape[0], 1, 1, 1, device=x.device) < self.survival_prob
        )
        return torch.div(x, self.survival_prob) * binary_tensor

    def forward(self, inputs):
        x = self.expand_conv(inputs) if self.expand else inputs
        x = self.conv(x)
        x = self.cbam(x)
        if self.use_residual:
            return self.stochastic_depth(x) + inputs
        else:
            return x

In [20]:
class EfficientNet(nn.Module):
    def __init__(self, version, num_classes):
        super(EfficientNet, self).__init__()
        width_factor, depth_factor, dropout_rate = self.calculate_factors(version)
        last_channels = ceil(1280 * width_factor)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.features = self.create_features(width_factor, depth_factor, last_channels)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(last_channels, num_classes),
        )

    def calculate_factors(self, version, alpha=1.2, beta=1.1):
        phi, res, drop_rate = phi_values[version]
        depth_factor = alpha**phi
        width_factor = beta**phi
        return width_factor, depth_factor, drop_rate

    def create_features(self, width_factor, depth_factor, last_channels):
        channels = int(32 * width_factor)
        features = [CNNBlock(3, channels, 3, stride=2, padding=1)]
        in_channels = channels

        for expand_ratio, channels, repeats, stride, kernel_size in base_model:
            out_channels = 4 * ceil(int(channels * width_factor) / 4)
            layers_repeats = ceil(repeats * depth_factor)

            for layer in range(layers_repeats):
                features.append(
                    InvertedResidualBlock(
                        in_channels,
                        out_channels,
                        expand_ratio=expand_ratio,
                        stride=stride if layer == 0 else 1,
                        kernel_size=kernel_size,
                        padding=kernel_size // 2,
                    )
                )
                in_channels = out_channels

        features.append(
            CNNBlock(in_channels, last_channels, kernel_size=1, stride=1, padding=0)
        )

        return nn.Sequential(*features)

    def forward(self, x):
        x = self.pool(self.features(x))
        return self.classifier(x.view(x.shape[0], -1))

In [21]:
def process_single_image(image_path):
    """Process a single image and ensure it's valid."""
    try:
        with Image.open(image_path) as img:
            # First verify the image
            img.verify()
        
        # Reopen to process (verify closes the image)
        with Image.open(image_path) as img:
            # Convert to RGB and save
            img = img.convert('RGB')
            img.save(image_path)
        return True
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return False
    
def remove_corrupted_images(directory):
    """Remove corrupted images and ensure all images are in RGB format."""
    removed_count = 0
    processed_count = 0
    
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(root, file)
                try:
                    if process_single_image(file_path):
                        processed_count += 1
                    else:
                        os.remove(file_path)
                        removed_count += 1
                except Exception as e:
                    print(f"Failed to process {file_path}: {e}")
                    try:
                        os.remove(file_path)
                        removed_count += 1
                    except:
                        print(f"Failed to remove corrupted file: {file_path}")
    
    print(f"Processed {processed_count} images")
    print(f"Removed {removed_count} corrupted images")

In [22]:
def create_train_test_split(data_dir, output_dir, test_size=0.2):
    """Create train-test split with enhanced error handling."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    train_dir = os.path.join(output_dir, "train")
    test_dir = os.path.join(output_dir, "test")

    for crop_type in os.listdir(data_dir):
        crop_path = os.path.join(data_dir, crop_type)
        if not os.path.isdir(crop_path):
            continue

        for disease in os.listdir(crop_path):
            try:
                disease_dir = os.path.join(crop_path, disease)
                if not os.path.isdir(disease_dir):
                    continue

                # Create output directories
                os.makedirs(os.path.join(train_dir, crop_type, disease), exist_ok=True)
                os.makedirs(os.path.join(test_dir, crop_type, disease), exist_ok=True)

                # Get valid images
                valid_images = []
                for img in os.listdir(disease_dir):
                    if img.lower().endswith(('.jpg', '.jpeg', '.png')):
                        img_path = os.path.join(disease_dir, img)
                        try:
                            with Image.open(img_path) as im:
                                # Verify and convert to RGB
                                im = im.convert('RGB')
                                valid_images.append(img_path)
                        except Exception as e:
                            print(f"Skipping invalid image: {img_path}, Error: {e}")

                if not valid_images:
                    print(f"No valid images found in {disease_dir}")
                    continue

                # Split and copy
                train_images, test_images = train_test_split(valid_images, test_size=test_size, random_state=42)

                for img in train_images:
                    try:
                        shutil.copy2(img, os.path.join(train_dir, crop_type, disease))
                    except Exception as e:
                        print(f"Error copying {img}: {e}")

                for img in test_images:
                    try:
                        shutil.copy2(img, os.path.join(test_dir, crop_type, disease))
                    except Exception as e:
                        print(f"Error copying {img}: {e}")

            except Exception as e:
                print(f"Error processing disease {disease}: {e}")
                continue


In [23]:
def train_efficientnet(data_dir, version="b3", num_classes=None, batch_size=16, epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    transform = transforms.Compose([
        transforms.Resize((phi_values[version][1], phi_values[version][1])),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    try:
        train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, "train"), transform=transform)
        test_dataset = datasets.ImageFolder(root=os.path.join(data_dir, "test"), transform=transform)
        
        if len(train_dataset) == 0 or len(test_dataset) == 0:
            raise ValueError("Empty datasets detected")

        print(f"Found {len(train_dataset)} training images and {len(test_dataset)} test images")
        
        if num_classes is None:
            num_classes = len(train_dataset.classes)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

        model = EfficientNet(version=version, num_classes=num_classes).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1)

        # ✅ Corrected GradScaler initialization
        scaler = torch.amp.GradScaler('cuda')

        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            for i, (inputs, labels) in enumerate(train_loader):
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()

                # ✅ Corrected autocast usage
                with torch.amp.autocast('cuda'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                running_loss += loss.item()
                if i % 10 == 0:  # Print more frequently
                    print(f"Epoch [{epoch + 1}/{epochs}], Batch [{i}/{len(train_loader)}], Loss: {loss.item():.4f}")

            epoch_loss = running_loss / len(train_loader)
            print(f"Epoch [{epoch + 1}/{epochs}], Average Loss: {epoch_loss:.4f}")
            scheduler.step(epoch_loss)

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Test Accuracy: {accuracy:.2f}%")
        
        torch.save(model.state_dict(), 'efficientnet_cbam_model.pth')
        
    except Exception as e:
        print(f"Training failed: {e}")
        raise


In [ ]:
# Main execution
raw_data_dir = "Raw Data"
processed_data_dir = "Processed_Data"

# remove_corrupted_images(raw_data_dir)
# create_train_test_split(raw_data_dir, processed_data_dir)
# remove_corrupted_images(processed_data_dir)
train_efficientnet(processed_data_dir)    

In [ ]:
import torch
import torch.nn as nn
import os
import json
from torch.utils.data import DataLoader
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from PIL import Image
from collections import defaultdict
import numpy as np

# Crop-to-disease mapping (you should define this based on your dataset)
crop_to_diseases = {
    'Cashew': ['Healthy', 'Disease1', 'Disease2'],
    'Cassava': ['Healthy', 'Disease3', 'Disease4'],
    'Maize': ['Healthy', 'Disease5', 'Disease6'],
    # Add all the crops and their respective diseases here
}

class CropDiseaseDataset(torch.utils.data.Dataset):
    def __init__(self, crop_name, data_dir, transform=None):
        """
        Args:
            crop_name (str): The crop name (Cashew, Cassava, etc.)
            data_dir (str): Path to the root data directory.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.crop_name = crop_name
        self.data_dir = data_dir
        self.transform = transform
        
        # Load diseases associated with this crop
        self.diseases = crop_to_diseases.get(crop_name, [])
        
        # Create a mapping from disease name to index (for classification)
        self.disease_to_idx = {disease: idx for idx, disease in enumerate(self.diseases)}
        
        # Initialize the image file paths and labels
        self.image_paths = []
        self.labels = []
        
        for disease in self.diseases:
            disease_dir = os.path.join(data_dir, crop_name, disease)
            for img_name in os.listdir(disease_dir):
                if img_name.endswith(".jpg") or img_name.endswith(".png"):
                    self.image_paths.append(os.path.join(disease_dir, img_name))
                    self.labels.append(self.disease_to_idx[disease])
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        # Load and preprocess the image
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        
        return image, label

def train_efficientnet(crop_name, data_dir, batch_size=32, epochs=10):
    # Define data transformations (e.g., normalization, resizing)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    # Load dataset specific to the crop
    dataset = CropDiseaseDataset(crop_name, data_dir, transform=transform)
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Initialize the EfficientNet model
    model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=len(dataset.diseases))

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    # Train the model
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}, Accuracy: {100*correct/total}%")

    # Save the trained model
    torch.save(model.state_dict(), f'{crop_name}_efficientnet.pth')

    # Save the disease-to-idx mapping for later use
    with open(f'{crop_name}_disease_to_idx.json', 'w') as f:
        json.dump(dataset.disease_to_idx, f)

    print("Training complete!")

# Prediction function for user input
def predict_disease(crop_name, model, image_path):
    # Load the disease-to-idx mapping for the selected crop
    with open(f'{crop_name}_disease_to_idx.json', 'r') as f:
        disease_to_idx = json.load(f)
    
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension
    
    # Predict the disease
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted_idx = torch.max(outputs, 1)
        predicted_disease = list(disease_to_idx.keys())[predicted_idx.item()]
    
    return predicted_disease

# Example Usage:
# First, train the model for a specific crop (e.g., "Cashew")
train_efficientnet('Cashew', 'Processed_Data', batch_size=32, epochs=10)

# Then, make a prediction for a specific crop and image
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=3)  # Adjust num_classes for your dataset
model.load_state_dict(torch.load('Cashew_efficientnet.pth'))

image_path = 'path_to_image.jpg'
predicted_disease = predict_disease('Cashew', model, image_path)
print(f"Predicted Disease: {predicted_disease}")


In [3]:
from efficientnet_pytorch import EfficientNet